<a href="https://colab.research.google.com/github/nightstorm0909/ChessAI/blob/master/ChessAI_train_Keras_Model_with_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!cd "/content/drive/My Drive/Colab Notebooks/ChessAI"
!ls

drive  sample_data


In [0]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dense, Flatten
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import clear_session

In [0]:
data = np.load(os.path.join("/content/drive/My Drive/Colab Notebooks/ChessAI/processed", "dataset_keras_4000_348821.npz"))
state_array = data['arr_0']
state_array = np.moveaxis(state_array, 1, 3)
policy_array = data['arr_1']
value_array = data['arr_2']
print("[INFO] Training data: State: {}, Policy: {}, Value: {}".format(state_array.shape, policy_array.shape, value_array.shape))


[INFO] Training data: State: (348821, 8, 8, 18), Policy: (348821, 1968), Value: (348821,)


In [0]:
def build_residual_block(x, index):
  in_x = x
  res_name = "res"+str(index)

  x = Conv2D(filters = 256, kernel_size = 3, padding = "same", 
           kernel_regularizer = l2(1e-4), name = res_name + "conv1_3x3_256")(x)
  x = BatchNormalization(axis = -1, name = res_name + "_batchnorm1")(x)
  x = Activation("relu",name = res_name + "_relu1")(x)

  x = Conv2D(filters = 256, kernel_size = 3, padding = "same",
           kernel_regularizer = l2(1e-4), name = res_name + "conv2_3x3_256")(x)
  x = BatchNormalization(axis = -1, name = res_name + "_batchnorm2")(x)
  x = Add(name = res_name + "_add")([in_x, x])
  x = Activation("relu",name = res_name + "_relu2")(x)
  return x

input_x = x = Input(shape = (8, 8, 18))
x = Conv2D(filters = 256, kernel_size = 5, padding = "same", kernel_regularizer = l2(1e-4), name = "input_conv_5x5_256")(x)
x = BatchNormalization(axis = -1, name = "input_batchnorm")(x)
x = Activation("relu", name="input_relu")(x)

for i in range(5):
  x = build_residual_block(x, i + 1)
res_out = x

# for policy output
x = Conv2D(filters = 2, kernel_size = 1, kernel_regularizer = l2(1e-4), name = "policy_conv_1x1_2")(res_out)
x = BatchNormalization(axis = -1, name = "policy_batchnorm")(x)
x = Activation("relu", name = "policy_relu")(x)
x = Flatten(name = "policy_flatten")(x)
policy_out = Dense(units = 1968, activation = "softmax", kernel_regularizer = l2(1e-4), name="policy_out")(x)

# for value output
x = Conv2D(filters = 4, kernel_size = 1, kernel_regularizer = l2(1e-4), name = "policy_conv_1x1_4")(res_out)
x = BatchNormalization(axis = -1, name = "value_batchnorm")(x)
x = Activation("relu", name = "value_relu")(x)
x = Flatten(name = "value_flatten")(x)
x = Dense(units = 256, kernel_regularizer = l2(1e-4), activation = "relu", name = "value_dense")(x)
value_out = Dense(units = 1, kernel_regularizer = l2(1e-4), activation = "tanh", name = "value_out")(x)

model = Model(inputs = input_x, outputs = [policy_out, value_out], name = "Chess Model")

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8, 8, 18)     0                                            
__________________________________________________________________________________________________
input_conv_5x5_256 (Conv2D)     (None, 8, 8, 256)    115456      input_1[0][0]                    
__________________________________________________________________________________________________
input_batchnorm (BatchNormaliza (None, 8, 8, 256)    1024        input_conv_5x5_256[0][0]         
__________________________________________________________________________________________________
input_relu (Activation)         (None, 8, 8, 256)    0           input_batchnorm[0][0]            
__________________________________________________________________________________________________
res1conv1_

In [0]:
import time

epochs = 10
batch_size = 256

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy = tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

tpu_model.compile(
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3, ),
    loss = [tf.keras.losses.categorical_crossentropy, tf.keras.losses.mean_squared_error],
    loss_weights = [1.25, 1.0]
)
start = time.time()

history = tpu_model.fit(
    x = state_array,
    y = [policy_array, value_array],
    batch_size = batch_size,
    epochs = epochs,
    shuffle = True, 
    validation_split=0.02
)
print("\n\n[INFO] Training completed in {:.04} hrs".format((time.time() - start) / 3600))

cpu_model = tpu_model.sync_to_cpu()
cpu_model.save_weights(os.path.join("/content/drive/My Drive/Colab Notebooks/ChessAI/nets", "keras_model_weights_tpu_{}e.h5".format(epochs)))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.17.125.162:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 7207861141904379785)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 13816945816331221900)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 15181982555864580038)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9832806244918732211)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5238464885113172451)
INFO:tensorflow:*** Available Device: _Devic

In [0]:
os.environ['COLAB_TPU_ADDR']

'10.17.125.162:8470'

In [0]:
cpu_model.save_weights(os.path.join("/content/drive/My Drive/Colab Notebooks/ChessAI/nets", "keras_model_weights_tpu_{}e.h5".format(epochs)))